# OpenCV Tutorial: A Guide to Learn OpenCV

This OpenCV tutorial is for beginners just getting started learning the basics. Inside this guide, you’ll learn basic image processing operations using the OpenCV library using Python.

# Loading and Displaying an image

In [1]:
# Import the necessary packages
import cv2
import imutils

In [2]:
# load the input image and show its dimensions, keeping in mind 
# that images are represented as a multi-dimensional Numpy array
# with shape no. rows (height) X no. columns (width) X no. channels (depth)
image = cv2.imread('jp.png')
(h, w, d) = image.shape
print("width = {}, height = {}, depth = {}".format(w, h, d))

width = 600, height = 322, depth = 3


In [3]:
# display the image 

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()        

## Accessing individual pixels

**What is `Pixel` :** All images consist of pixels which are the raw building blocks of images. Images are made of pixels in a grid. A 640 x 480 image has 640 columns (the width) and 480 rows (the height). There are 640 * 480 = 307200  pixels in an image with those dimensions.


In OpenCV color images in the RGB (Red, Green, Blue) color space have a 3-tuple associated with each pixel: `(B, G, R)`

Notice the ordering is `BGR` rather than `RGB`. This is because when OpenCV was first being developed many years ago the standard was BGR ordering. Over the years, the standard has now become RGB but OpenCV still maintains this “legacy” BGR ordering to ensure no existing code breaks.

Each value in the BGR 3-tuple has a range of `[0, 255]` . How many color possibilities are there for each pixel in an RGB image in OpenCV? That’s easy: `256 * 256 * 256 = 16777216 `.

In [4]:
# access the RGB pixel
(B, G, R) = image[100, 50]
print("R = {}, G = {}, B = {}".format(R, G, B))

R = 41, G = 49, B = 37


## Array slicing and cropping

Extracting ***“regions of interest” (ROIs)*** is an important skill for image processing

Say, for example, you’re working on recognizing faces in a movie. First, you’d run a face detection algorithm to find the coordinates of faces in all the frames you’re working with. Then you’d want to extract the face ROIs and either save them or process them. Locating all frames containing Dr. Ian Malcolm in Jurassic Park would be a great face recognition mini-project to work on.

***For now, let’s just manually extract an ROI. This can be accomplished with array slicing.***

**Array slicing** :  `image[startY:endY, startX:endX]`

In [5]:
# extract a 100X100 pixel square ROI from the input image 
# starting at x=320,y=60 and ending at x=420,y=160

roi = image[60:160 , 320:420 ]
cv2.imshow("ROI", roi)
cv2.waitKey(0)
cv2.destroyAllWindows()        

## Resizing images

Resizing images is important for a number of reasons. First, you might want to resize a large image to fit on your screen. Image processing is also faster on smaller images because there are fewer pixels to process. In the case of deep learning, we often resize images, ignoring aspect ratio, so that the volume fits into a network which requires that an image be square and of a certain dimension.

**Let’s resize our original image to 200 x 200 pixels:**

In [6]:
# resizing image to 200X200px, ignoring aspect ratio
resized = cv2.resize(image, (200, 200))
cv2.imshow("Fixed Resizing", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Let’s calculate the aspect ratio of the original image and use it to resize an image so that it doesn’t appear squished and distorted:

In [7]:
# fixed resizing and distort aspect ratio so let's resize the width
# to be 300px but compute the new height based on the aspect ratio

r = 300.0 / w
dim = (300, int(h * r))
resized = cv2.resize(image, dim)
cv2.imshow("Aspect Ratio Resize", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Feeding `dim`  (our dimensions) into the `cv2.resize`  function, we’ve now obtained a new image named `resized`  which is not distorted

**But can we make this process of preserving aspect ratio during resizing even easier?**

YES!

Computing the aspect ratio each time we want to resize an image is a bit tedious, so I wrapped the code in a function within `imutils` .

Here is how you may use `imutils.resize` :

All you need to provide is your **target `width`**  or **target `height`**  as a keyword argument 

In [9]:
# computing aspect ratio using imutils library
resized = imutils.resize(image, width = 300)
cv2.imshow("Imutils Resize", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Rotating an image

Rotating an image with OpenCV about the center point requires three steps:
* compute the center point using the image width and height,
* compute a rotation matrix with `cv2.getRotationMatrix2D`,
* use the rotation matrix to warp the image with `cv2.warpAffine.`


In [10]:

## Let's rotate an image 45 degrees clockwise using OpenCV by first
## computing the image center, then constructing the rotation matrix,
## and then finally applying the affine wrap

center = (w // 2, h // 2) # // --> No floating pt. values
M = cv2.getRotationMatrix2D(center, -45, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [11]:
# Rotation can also be easily accomplished via imutils with less code

rotated = imutils.rotate(image, -45)
cv2.imshow("Imutils Rotation", rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
## OpenCV doesn't "care" if our rotated image is clipped after rotation
## so we can instead use another imutils convenience function to help
## us out..

rotated = imutils.rotate_bound(image, 45)
cv2.imshow("Imutils Bound Rotation", rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Writng function for `imshow()` to avoid writing same code again

In [13]:
def show(image):
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

## Smoothing an Image

In many image processing pipelines, we must blur an image to reduce high-frequency noise, making it easier for our algorithms to detect and understand the actual contents of the image rather than just noise that will “confuse” our algorithms. Blurring an image is very easy in OpenCV and there are a number of ways to accomplish it.

In [15]:
# apply a Gaussian blur with a 11*11 kernel to the image to smooth it,
# useful when reducing high frequency noise

blurred = cv2.GaussianBlur(image, (11,11), 0)
show(blurred)

## Drawing on an image

In this section, we’re going to draw a rectangle, circle, and line on an input image. We’ll also overlay text on an image as well.

Before we move on with drawing on an image with OpenCV, take note that ***drawing operations on images are performed in-place***. Therefore at the beginning of each code block, we make a copy of the original image storing the copy as `output` . We then proceed to draw on the image called `output` in-place so we do not destroy our original image.

Since we are using OpenCV’s functions rather than NumPy operations ***we can supply our coordinates in `(x, y)` order rather than `(y, x)`*** since we are not manipulating or accessing the NumPy array directly — OpenCV is taking care of that for us.

In [22]:
## Let's draw a rectangle
# draw a 2px thick red rectangle surrounding the face
output = image.copy()
cv2.rectangle(output, (320, 60), (420, 160), (0, 0, 255), 2)
show(output)

Drawing rectangles in OpenCV couldn’t be any easier. Using pre-calculated coordinates, I’ve supplied the following parameters to the `cv2.rectangle`

* `img`: The destination image to draw upon. We’re drawing on `output`.
* `pt1`: Our starting pixel coordinate which is the top-left. In our case, the top-left is `(320, 60)`.
* `pt2`: The ending pixel — bottom-right. The bottom-right pixel is located at `(420, 160)`.
* `color`: BGR tuple. To represent red, I’ve supplied `(0 , 0, 255)`.
* `thickness`: Line thickness (a negative value will make a solid rectangle). I’ve supplied a thickness of `2` .



In [23]:
# draw a blue 20px (filled in) circle on the image centered at
# x = 300, y = 150

output = image.copy()
cv2.circle(output, (300, 150), 20, (255, 0, 0), -1)
show(output)

To draw a circle, you need to supply following parameters to `cv2.circle` :

* `img`: The output image.
* `center`: Our circle’s center coordinate. I supplied `(300, 150)`  which is right in front of Ellie’s eyes.
* `radius`: The circle radius in pixels. I provided a value of `20`  pixels.
* `color`: Circle color. This time I went with blue as is denoted by 255 in the B and 0s in the G + R components of the BGR tuple, `(255, 0, 0)` .
* `thickness`: The line thickness. Since I supplied a negative value `(-1 )`, the circle is solid/filled in.

In [27]:
# draw a 5px thick line from x=60, y=20 to x=400, y=200

output = image.copy()
cv2.line(output, (60, 20), (400, 200), (0, 0, 255), 5)
show(output)

Just as in a rectangle, we supply two points, a color, and a line thickness. OpenCV’s backend does the rest.

## Writing Text

Oftentimes you’ll find that you want to overlay text on an image for display purposes. If you’re working on face recognition you’ll likely want to draw the person’s name above their face. Or if you advance in your computer vision career you may build an image classifier or object detector. In these cases, you’ll find that you want to draw text containing the class name and probability.

Let’s see how OpenCV’s `putText()` function works:

In [33]:
## Writing Text

# draw green text on the image 

output = image.copy()
cv2.putText(output, "OpenCV + Jurassic Park!!!", (10, 25),
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
show(output)

The `putText`  function of OpenCV is responsible for drawing text on an image. Let’s take a look at the required parameters:

* `img` : The output image.
* `text` : The string of text we’d like to write/draw on the image.
* `pt` : The starting point for the text.
* `font`: uses the `cv2.FONT_HERSHEY_SIMPLEX`.You can see the available `fonts` by pressing `Tab` key while writing font.
* `scale`: Font size multiplier.
* `color`: Text Color.
* `thickness`: The thickness of the stroke in pixels.


## `Continue in Part-2` !